In [1]:
import random
import numpy as np
import tensorflow as tf
from IPython.display import clear_output
k = tf.keras

In [2]:
k.backend.clear_session()

In [3]:
pg = k.Sequential([
    k.layers.Input(shape=[4]),
    k.layers.Dense(units=16,activation='relu'),
    k.layers.Dense(units=16,activation='relu'),
    k.layers.Dense(units=2,activation='softmax')
])

2021-10-28 18:59:46.388413: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
pg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________


In [5]:
def get_action(obs):
    probs = pg(obs)[0].numpy()
    probs[-1] = 1-np.sum(probs[:-1]) #handle floating point error
    action = np.random.choice([0,1], p=probs)
    return action

def compute_loss(states, actions, returns):
    probs = pg(states)
    indices = [(i,j) for i,j in enumerate(actions)]
    
    actions_prob = tf.gather_nd(probs,indices)
    logp = tf.math.log(actions_prob)
    return -tf.reduce_mean(logp * returns)

In [6]:
optimizer = k.optimizers.Adam(0.003)

In [7]:
MAX_ITER = 1000 
EPISODES = 1000
DISCOUNT = 0.99

In [8]:
import gym
env = gym.make('CartPole-v1')

In [9]:
history = {
    'losses': [],
    'rewards': [],
}

In [10]:
for e in range(EPISODES):
    done = False
    state = env.reset()
    states = []
    actions = []
    rewards = []
    
    i = 0
    while not done and i < MAX_ITER:
        states.append(state.copy())
        
        action = get_action(state.reshape((1,-1)))
        actions.append(action)
        
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        
        state = next_state
        i+=1
    
    # Calculated the discounted reward
    # Reward in the future is less valueable than the beginning
    discounted_rewards = []
    total_r = 0
    for reward in reversed(rewards):
        total_r = reward + total_r ** DISCOUNT
        discounted_rewards.append(total_r)
    
    discounted_rewards = np.array(discounted_rewards)[::-1]
    
    with tf.GradientTape() as tape:
        loss = compute_loss(np.array(states), np.array(actions), discounted_rewards)
    
    grads = tape.gradient(loss, pg.trainable_weights)
    optimizer.apply_gradients(zip(grads, pg.trainable_weights))
    
    history['losses'].append(loss)
    history['rewards'].append(sum(rewards))
    print(f'Ep {e+1}: loss = {loss:.04f}, reward= {sum(rewards)}')
    clear_output(wait=True)

Ep 1000: loss = 7.7158, reward= 42.0


In [11]:
import matplotlib.pyplot as plt

In [12]:
fig = plt.figure(figsize=[16,5])
plt.plot(history['rewards'])

In [13]:
scores = []
for e in range(100):
    state = env.reset()
    rewards = []
    done = False
    while not done:
        action = get_action(state.reshape(1,-1))
        state, reward, done, _ = env.step(action)
        rewards.append(reward)
    scores.append(sum(rewards))

In [15]:
np.max(scores)

126.0

In [14]:
np.mean(scores)

64.18

#### The problem with vanilla policy gradient is the variance of the input is really high